In [ ]:
import compas_rrc as rrc
from compas.geometry import Point, Vector, Frame
import csv
import os



# Start ROS & ABB clients
ros = rrc.RosClient()
ros.run()

abb = rrc.AbbClient(ros, '/rob1')
print('Connected to ABB IRB 6700')
    


In [ ]:
done = abb.send_and_wait(rrc.SetTool('fingergrab'))
frame = abb.send_and_wait(rrc.GetFrame())
print("Active TCP frame:", frame)


In [ ]:
csv_filename = "pose_log.csv"
csv_path = os.path.join(os.getcwd(), csv_filename)

with open(csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["cycle", "direction", "x", "y", "z", "rx", "ry", "rz"])


In [ ]:
def open_gripper():
    abb.send_and_wait(rrc.SetDigital('ABB_Scalable_IO_0_DO8', 0))
    abb.send_and_wait(rrc.SetDigital('ABB_Scalable_IO_0_DO6', 1))
    print(" → Gripper opened")

def close_gripper():
    # abb.send_and_wait(rrc.WaitTime(1.0))
    abb.send_and_wait(rrc.SetDigital('ABB_Scalable_IO_0_DO6', 0))
    abb.send_and_wait(rrc.SetDigital('ABB_Scalable_IO_0_DO8', 1))
    # wait 2 seconds on the robot controller
    # abb.send_and_wait(rrc.WaitTime(1.0))
    print(" → Gripper closed")

# Define frames with matching keys
frames = {
    "home":           Frame(Point(690.435653, -1306.975386, 1366.823567), Vector(0,1,0), Vector(1,0,0)),
    "safe_plane":     Frame(Point(-128.61, -1422.05,  1375.45), Vector(-0.999299,0.03518,0.012813), Vector(0.035444,0.99915,0.021035)),
    "approach_pick":  Frame(Point(451.48, -1754.92, 1242.00), Vector(0.999962,-0.007326,0.004801), Vector(-0.007193,-0.999608,-0.027055)),
    "pick":           Frame(Point(451.48, -1754.92, 1192.00), Vector(0.999962,-0.007326,0.004801), Vector(-0.007193,-0.999608,-0.027055)),
    "approach_place": Frame(Point(444.41, -1754.95, 1243.00), Vector(0.999962,-0.007326,0.004801), Vector(-0.007193,-0.999608,-0.027055)),
    "place":          Frame(Point(444.41, -1754.95, 1193.22), Vector(0.999962,-0.007326,0.004801), Vector(-0.007193,-0.999608,-0.027055)),}

speed_fly    = 500  # mm/s for transit moves
speed_action =  20  # mm/s for pick/place
zone         = rrc.Zone.FINE

# add z offset to frames for the place position
frames["place"].point.z += 25
frames["approach_place"].point.z += 28


# for key, fm in frames.items():
#     frames[key] = Frame(fm.point + Vector(0, 0, 100), fm.xaxis, fm.yaxis)


def do_move(step_name, frame, speed, motion):
    abb.send_and_wait(rrc.MoveToFrame(frame, speed, zone, motion))
    print(f" → Step '{step_name}' ({motion}) completed")
    


In [ ]:
do_move("Hitting home",    frames["home"], speed_fly,    rrc.Motion.JOINT)

In [ ]:
open_gripper()
abb.send_and_wait(rrc.WaitTime(2.0))
close_gripper()

In [ ]:
# Sequential workflow
def pick_and_place(pick_approach, pick, place_approach, place, cycle_num, direction):
    print(f"\n[CYCLE {cycle_num}] {direction}: {pick} → {place}")
    do_move(f"Approach {pick}",   frames[pick_approach],  speed_fly,    rrc.Motion.JOINT)
    open_gripper()
    do_move(f"Pick {pick}",       frames[pick],           speed_action, rrc.Motion.LINEAR)
    close_gripper()
    do_move(f"Retract {pick}",    frames[pick_approach],  speed_action, rrc.Motion.LINEAR)
    do_move(f"Approach {place}",  frames[place_approach], speed_fly,    rrc.Motion.JOINT)
    do_move(f"Place {place}",     frames[place],          speed_action, rrc.Motion.LINEAR)
    
    log_pose_to_csv(cycle_num, direction)

    abb.send_and_wait(rrc.WaitTime(1.0))
    open_gripper()
    abb.send_and_wait(rrc.WaitTime(1.0))
    do_move(f"Retract {place}",   frames[place_approach], speed_action, rrc.Motion.LINEAR)

print("All steps completed. Shutting down.")
ros.close()


In [ ]:
do_move("Approach place",  frames["approach_place"], speed_fly,    rrc.Motion.JOINT)
do_move("Place",           frames["place"],          speed_action, rrc.Motion.LINEAR)